In [1]:
import numpy as np
from scipy.spatial import Delaunay
import torch
import torch.sparse as sparse
import matplotlib.pyplot as plt

In [2]:
def torch_compute_triangle_area_vectors(verts, tris):
    return 0.5*torch.cross(verts[tris[:, 1],:] - verts[tris[:, 0],:],verts[tris[:, 2],:] - verts[tris[:, 0],:], dim=-1)
def torch_compute_triangle_normals(verts, tris, return_areas = False):
    area_vecs = torch_compute_triangle_area_vectors(verts, tris)
    areas = torch.linalg.norm(area_vecs, dim=-1)
    if(return_areas):
        return area_vecs/areas[:,None], areas
    else:
        return area_vecs/areas[:,None]
def torch_edgelist(tris: torch.Tensor):
    n = tris.shape[0]

    # Collect edges per triangle in consistent order
    e01 = tris[:, [0, 1]]
    e12 = tris[:, [1, 2]]
    e20 = tris[:, [2, 0]]
    edges_per_tri = torch.stack([e01, e12, e20], dim=1)  # (n,3,2)
    flat_edges = edges_per_tri.reshape(-1, 2)            # (3n,2)

    # Undirected version (for uniqueness)
    sorted_edges, sort_idx = torch.sort(flat_edges, dim=1)

    # Unique edges + inverse mapping
    unique_edges, inv, counts = torch.unique(
        sorted_edges, return_inverse=True, return_counts=True, dim=0
    )

    # Per-triangle edge indices
    tri_to_edge = inv.view(n, 3)

    # Orientation: +1 if flat_edges == unique_edges, -1 if reversed
    edge_from_unique = unique_edges[inv]
    orientation = torch.where(
        torch.all(flat_edges == edge_from_unique, dim=1),
        torch.ones_like(inv),
        -torch.ones_like(inv)
    ).view(n, 3)

    return unique_edges, tri_to_edge, orientation, counts
def torch_midpoint_edge_normals(tri_normals, edgelist, tri_to_edge):
    edge_normals = torch.zeros((edgelist.shape[0],3))
    edge_normals[tri_to_edge[:,0],:] += tri_normals
    edge_normals[tri_to_edge[:,1],:] += tri_normals
    edge_normals[tri_to_edge[:,2],:] += tri_normals
    edge_normals /= torch.linalg.norm(edge_normals, dim=1, keepdim=True)
    return edge_normals
def torch_ffm(verts, tris):
    device, dtype = verts.device, verts.dtype
    a = torch.zeros((tris.shape[0],2,2),  device=device, dtype=dtype)
    v_ijk = verts[tris]
    a[:,0,0] = torch.sum((v_ijk[:,1,:] - v_ijk[:, 0, :])**2.0, dim=-1)
    a[:,1,1] = torch.sum((v_ijk[:,2,:] - v_ijk[:, 0, :])**2.0, dim=-1)
    a[:,0,1] = torch.sum((v_ijk[:,1,:] - v_ijk[:, 0, :])*(v_ijk[:,2,:] - v_ijk[:, 0, :]), dim=-1)
    a[:,1,0] = a[:,0,1]
    return a
def torch_sfm(verts, tris, tri_to_edge, edge_normals):
    device, dtype = verts.device, verts.dtype
    F = tris.shape[0]

    # Vertex positions per triangle
    Vi = verts[tris[:, 0]]  # (F,3)
    Vj = verts[tris[:, 1]]  # (F,3)
    Vk = verts[tris[:, 2]]  # (F,3)

    # Map opposite-edge normals to vertices of each triangle.
    # tri_to_edge is ordered [e01, e12, e20] = [(i,j),(j,k),(k,i)]
    e01 = tri_to_edge[:, 0]   # edge (i,j) -> opposite to vertex k
    e12 = tri_to_edge[:, 1]   # edge (j,k) -> opposite to vertex i
    e20 = tri_to_edge[:, 2]   # edge (k,i) -> opposite to vertex j

    n_i = edge_normals[e12]   # opposite edge to i is (j,k)
    n_j = edge_normals[e20]   # opposite edge to j is (k,i)
    n_k = edge_normals[e01]   # opposite edge to k is (i,j)

    # Differences
    nij = n_i - n_j           # (F,3)
    nik = n_i - n_k           # (F,3)
    vi_minus_vj = Vi - Vj     # (F,3)
    vi_minus_vk = Vi - Vk     # (F,3)

    # Dot products -> entries
    b00 = torch.sum(nij * vi_minus_vj, dim=-1)   # (F,)
    b01 = torch.sum(nik * vi_minus_vj, dim=-1)   # (F,)
    b10 = torch.sum(nij * vi_minus_vk, dim=-1)   # (F,)
    b11 = torch.sum(nik * vi_minus_vk, dim=-1)   # (F,)

    b = torch.empty((F, 2, 2), device=device, dtype=dtype)
    b[:, 0, 0] = b00
    b[:, 0, 1] = b01
    b[:, 1, 0] = b10
    b[:, 1, 1] = b11
    return b

In [3]:
w0 = 128
h0 = 64
dx = 0.5
N0 = int(h0*w0/(dx*dx))
np.random.seed(1101252)
vertices = np.random.rand(N0, 2)
vertices[:,0] *= h0
vertices[:,1] *= w0
triang = Delaunay(vertices)
triangles = torch.tensor(triang.simplices)
vertices = torch.tensor(np.concatenate([vertices, np.zeros((N0,1))], axis=1))

In [4]:
edgelist, tri_to_edge, orientation, counts = torch_edgelist(triangles)
tri_normals = torch_compute_triangle_normals(vertices, triangles, return_areas=False)
midpoint_edge_normals = torch_midpoint_edge_normals(tri_normals, edgelist, tri_to_edge)
a_matr = torch_ffm(vertices, triangles)
b_matr = torch_sfm(vertices, triangles, tri_to_edge, midpoint_edge_normals)